# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

cities = "output_data/cities.csv"
cities_df = pd.read_csv(cities)
cities_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,geraldton,-28.77,114.60,64.40,68,0,5.82,AU,1603326269
1,1,nikolskoye,59.70,30.79,39.20,93,90,8.95,RU,1603326231
2,2,east london,65.00,-15.00,23.00,92,20,4.70,IS,1603326205
3,3,yellowknife,62.46,-114.35,24.80,73,90,18.34,CA,1603326010
4,4,tiksi,71.69,128.87,10.53,94,100,5.19,RU,1603326080


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations= cities_df[["Lat", "Lng"]].astype(float)
weight = cities_df["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_weather=cities_df[cities_df["Max Temp"] >70][cities_df["Max Temp"]<90][cities_df["Wind Speed"]<15][cities_df["Cloudiness"]<20]
ideal_weather.head()

C:\Users\JMahon\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
C:\Users\JMahon\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
C:\Users\JMahon\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,hilo,19.73,-155.09,87.80,48,1,11.41,US,1603326007
36,36,gat,31.61,34.76,73.00,77,0,2.24,IL,1603326343
38,38,hithadhoo,-0.60,73.08,82.51,74,11,13.04,MV,1603326199
40,40,butaritari,3.07,172.79,81.97,71,3,9.62,KI,1603326141
41,41,grand river south east,38.88,-109.50,78.80,9,1,9.17,US,1603326058


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df=ideal_weather.loc[:, ["City", "Country", "Lat", "Lng"]]
hotel_df ["Hotel Name"]=""
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
8,hilo,US,19.73,-155.09,
36,gat,IL,31.61,34.76,
38,hithadhoo,MV,-0.60,73.08,
40,butaritari,KI,3.07,172.79,
41,grand river south east,US,38.88,-109.50,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# response = requests.get(query_url).json()
# print(json.dumps(response, indent=4, sort_keys=True))

In [8]:
# Set parameters to search for a hotel
params = {"radius": 5000,
    "types": "lodging",
    "key": g_key}

# Iterate through 
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f"{lat},{lng}"
    
    response = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, 'Hotel'] = response['results'][0]['name']
    except (IndexError):
        print("Missing")

Missing
Missing
Missing
Missing
Missing
Missing
Missing


In [9]:
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name,Hotel
8,hilo,US,19.73,-155.09,,Hilo Hawaiian Hotel
36,gat,IL,31.61,34.76,,OlusHome
38,hithadhoo,MV,-0.60,73.08,,Scoop Guest House
40,butaritari,KI,3.07,172.79,,Isles Sunset Lodge
41,grand river south east,US,38.88,-109.50,,NaN


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))